In [1]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# %matplotlib inline

plt.rc('font',  family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import mutual_info_classif

import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_addons as tfa

import transformers
from transformers import AutoTokenizer
from transformers import TFAutoModel
transformers.logging.set_verbosity_error()

import re
from glob import glob
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')


import argparse

# import wandb
# from wandb.keras import WandbCallback
# wandb.init(project='DACON_235978' , name='gnoeyheat')


parser = argparse.ArgumentParser(description='gnoeyheat')
parser.add_argument('--text_pretrained_model', default='roberta', type=str)
parser.add_argument('--text_len', default=300 , type=int)
parser.add_argument('--optimizer' , default='sgd' , type=str) # sgd or adam
parser.add_argument('--learning_rate', default=0.002, type=float)
parser.add_argument('--loss', default='cc', type=str) # cc or fl
parser.add_argument('--label_smoothing' ,default=0.1 , type=float)
parser.add_argument('--batch_size' , default=1 ,type=int )
parser.add_argument('--epochs', default=30, type=int)
parser.add_argument('--validation_split' , default=0.1 , type=float)
parser.add_argument('--seed', default=1011, type=int)
args = parser.parse_args('')

# wandb.config.update(args)

os.environ['CUDA_VISIBLE_DEVICES'] = "0"

text_pretrained_model = args.text_pretrained_model
text_len = args.text_len
BATCH_SIZE = args.batch_size
EPOCHS = args.epochs
VALIDATION_SPLIT=args.validation_split
SEED=args.seed

def set_seeds(seed=SEED):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)


set_seeds()


if args.text_pretrained_model == "roberta":
    text_pretrained_model = "klue/roberta-large"

tokenizer = AutoTokenizer.from_pretrained(text_pretrained_model)

# tokenizer.truncation_size = 'left'

train = pd.read_csv('./input/train.csv')
test = pd.read_csv('./input/test.csv')

X_txt = train['overview']
X_test_txt = test['overview']

y = train['cat3']
y_encoder = {key: value for key, value in zip(np.unique(y), range(len(np.unique(y))))}
y = np.array([y_encoder[k] for k in y ])

X_txt.shape,  X_test_txt.shape, y.shape

def text_cleaning(df):
    df = df.apply(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+','',x))
    df = df.apply(lambda x : ' '.join(x.split()))
    return df

X_txt = text_cleaning(X_txt)
X_test_txt = text_cleaning(X_test_txt)

X_txt.iloc[0]

'소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 올리고 있으며 바다낚시터로도 유명하다 항 주변에 설치된 양식장들은 섬사람들의 부지런한 생활상을 고스 란히 담고 있으며 일몰 때 섬의 정경은 바다의 아름다움을 그대로 품고 있는 듯하다 또한 섬에는 각시여 전설 도둑바위 등의 설화가 전해 내려오고 있으며 매년 정월 풍어제 풍속이 이어지고 있다'

In [2]:
from hanspell import spell_checker

def text_correcting(df):
    correct = []
    for text in tqdm(df):
        if len(text) < 500:
            correct.append(spell_checker.check(text).checked)
        else:
            temp = []
            for i in range(0, len(text) , 500):
                temp.append(spell_checker.check(text[i:i+500]).checked)
            correct.append(''.join(temp))
    return pd.Series(correct, name='overview')

In [4]:
spell_checker.check('이문장의맞추법을계산해보시오')#  생각보다 성능좋다 일일이 안해도 될만큼 , 역시나



Checked(result=True, original='이문장의맞추법을계산해보시오', checked='이 문장의 맞춤법을 계산해보시오', errors=1, words=OrderedDict([('이', 1), ('문장의', 1), ('맞춤법을', 1), ('계산해보시오', 1)]), time=0.024933576583862305)

In [6]:
spell_checker.check('이문장의맞추법을계산해보시오')

Checked(result=True, original='이문장의맞추법을계산해보시오', checked='이 문장의 맞춤법을 계산해보시오', errors=1, words=OrderedDict([('이', 1), ('문장의', 1), ('맞춤법을', 1), ('계산해보시오', 1)]), time=0.024933815002441406)

In [7]:
# 생각보다 성능 좋다
spell_checker.check('점문점이다')

Checked(result=True, original='점문점이다', checked='전문점이다', errors=1, words=OrderedDict([('전문점이다', 1)]), time=0.015956878662109375)

In [8]:
#X_txt = text_correcting(X_txt)
#77%|███████▋  | 13113/16986 [3:11:20<56:30,  1.14it/s]
#JSONDecodeError: Expecting value: line 1 column 1 (char 0) ... ?


In [10]:
X_txt

0        소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...
1        경기도 이천시 모가면에 있는 골프장으로 대중제 홀이다 회원제로 개장을 했다가 년 대...
2        금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...
3        철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다 경상북도 안동시에 있는 한식 ...
4             영업시간 대에 걸쳐 아귀만을 전문으로 취급하는 전통과 역사를 자랑하는 음식점이다
                               ...                        
16981               해발 에 자리한 식담겸 카페점문점이다곤드레밥과 감자전을 판매하고 있다
16982    설악힐호텔은 동해고속도로 속초톨게이트에서 멀지 않은 관광로 변에 있다 속초의 대표 ...
16983    충남 서산시 중심가에 위치한 줌모텔은 프라이버스가 보장되는 조용한 공간으로 가치가 ...
16984    토토큰바위캠핑장은 경기도 가평지역 내에서도 청정지역으로 손꼽히는 지역으로 주변에 화...
16985    포천의 진산으로 불리우는 왕방산에는 천년의 역사를 간직하고 있는 왕산사가 자리하고 ...
Name: overview, Length: 16986, dtype: object

In [11]:
#train['overview_correcting'] =  X_txt
#train.to_csv('train_overview_correcting.csv', index=False)
#X_test_txt = text_correcting(X_test_txt)
#test['overview_correcting'].to_csv('test _overview_correcting.csv', index=False)

In [12]:
train.to_csv('data/pp_train.csv', index=False)
test.to_csv('data/pp_test.csv', index=False)

In [13]:
X_txt.iloc[0]

'소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 올리고 있으며 바다낚시터로도 유명하다 항 주변에 설치된 양식장들은 섬사람들의 부지런한 생활상을 고스 란히 담고 있으며 일몰 때 섬의 정경은 바다의 아름다움을 그대로 품고 있는 듯하다 또한 섬에는 각시여 전설 도둑바위 등의 설화가 전해 내려오고 있으며 매년 정월 풍어제 풍속이 이어지고 있다'

In [14]:
train['len'] = train['overview'].apply(tokenizer.tokenize).apply(len)
test['len'] = test['overview'].apply(tokenizer.tokenize).apply(len)
train['len'].median() , test['len'].median()

(144.0, 144.0)

In [15]:
# tf.keras.utils.Sequence == torch.utils.data.Dataset
class DataGenerator(tf.keras.utils.Sequence):

    def __init__(
        self,
        sentence,
        labels,
        batch_size=BATCH_SIZE,
        shuffle=True,
        include_targets=True
    ):
        self.sentence = sentence
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        self.tokenizer = tokenizer
        self.indexes = np.arange(len(self.sentence))
        self.on_epoch_end()

    def __len__(self):
        return len(self.sentence) // self.batch_size

    def __getitem__(self, idx):
        indexes = self.indexes[idx * self.batch_size : (idx+1) * self.batch_size]
        sentence = self.sentence[indexes]

        encoded = self.tokenizer.batch_encode_plus(
            sentence.tolist(),
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            max_length =text_len,
            return_tensors='tf',
            return_token_type_ids=True,
            return_attention_mask=True
        )

        input_ids = np.array(encoded['input_ids'] , dtype='int32')
        attention_masks = np.array(encoded['attention_mask'], dtype='int32')
        token_type_ids = np.array(encoded['token_type_ids'],dtype='int32')

        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype='int32')
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids , attention_masks, token_type_ids]
    #on_epoch_end로 인해 epoch이 끈날때마다 shuffle 되는 듯하다

    def on_epoch_end(self):
        if self.shuffle:
            np.random.RandomState(SEED).shuffle(self.indexes)

X_train, X_val , y_train, y_val = train_test_split(X_txt, y , test_size=VALIDATION_SPLIT , random_state=SEED, stratify=y)


# to_categorical == one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train)
y_val = tf.keras.utils.to_categorical(y_val)

X_train.shape, X_val.shape , y_train.shape  , y_val.shape


train_ds = DataGenerator(
    X_train.values,y_train,
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_ds = DataGenerator(
    X_val.values, y_val,
    batch_size=BATCH_SIZE,
    shuffle=False
)




## Modeling

In [20]:
lr = 2e-2

In [21]:
input_ids = tf.keras.layers.Input(
    shape=(text_len,), dtype=tf.int32, name='input_ids'
)

attention_masks = tf.keras.layers.Input(
    shape=(text_len,) , dtype=tf.int32, name='attention_masks'
)

token_type_ids = tf.keras.layers.Input(
    shape=(text_len,), dtype=tf.int32, name='token_type_ids'
)


bert_model = TFAutoModel.from_pretrained(text_pretrained_model, from_pt=True)
bert_model.trainable =True

bert_output = bert_model(
    input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
)


x = bert_output.last_hidden_state
# GlobalAveragePooling1D 해당 레이어에서는 각 예시에 대해 sequence 차원을 평균하여 고정된 길이의 벡터를 출력한다.
# 이를 통해 가변적인 길이의 입력을 간단하게 처리할 수 있다.
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.2)(x)
# y_train.shape[1] one-hot encoding 된 차원을 따르는 것을 의미 y_
output = layers.Dense(y_train.shape[1], activation='softmax')(x)


# 그래프 생성
model = tf.keras.models.Model(
    inputs=[input_ids, attention_masks, token_type_ids] , outputs=output
)


if args.optimizer == 'sgd':
    optim = tf.keras.optimizers.SGD(
        learning_rate=lr, momentum=0.9
    )
if args.loss == 'cc':
    # categorical 이므로 원-핫 인코딩 문제
    loss_function = tf.keras.losses.CategoricalCrossentropy(
        label_smoothing=args.label_smoothing
    )

model.compile(

    optimizer=optim,
    loss= loss_function,
    metrics= tfa.metrics.F1Score(num_classes=y_train.shape[1], average='weighted')
)

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 300)]        0           []                               
                                                                                                  
 attention_masks (InputLayer)   [(None, 300)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 300)]        0           []                               
                                                                                                  
 tf_roberta_model_1 (TFRobertaM  TFBaseModelOutputWi  336656384  ['input_ids[0][0]',              
 odel)                          thPooling(last_hidd               'attention_masks[0][0]',  

## Training

In [22]:
checkpoint_path = f"load_model/{parser.description}"


def scheduler(epoch, lr):
    if epoch < 20:
        return lr
    else:
        return lr * tf.math.exp(-0.1)



callback = [
    tf.keras.callbacks.LearningRateScheduler(scheduler),
    tf.keras.callbacks.ModelCheckpoint(
        checkpoint_path,
        monitor='val_f1_score',
        save_best_only=True,
        save_weights_only=True,
        mode='max',
    )
]

In [23]:
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    callbacks= [callback],
    validation_data =val_ds
)

acc = history.history['f1_score']
val_acc = history.history['val_f1_score']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(acc, label='Training Weighted-F1')
plt.plot(val_acc, label='Validation Weighted-F1')
plt.legend(loc='lower right')
plt.title("Training and Validation Weighted-F1")
plt.show()
# 1epoch 내내 8프로여도 설계한대로 여서 30epoch에 0.85까지 달성한다는 그런 이유와 계획을 봐야할듯

"""
WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
WARNING:tensorflow:The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
WARNING:tensorflow:Gradients do not exist for variables ['tf_roberta_model/roberta/pooler/dense/kernel:0', 'tf_roberta_model/roberta/pooler/dense/bias:0'] when minimizing the loss. If you're using `model.compile()`, did you forget to provide a `loss`argument?
WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
WARNING:tensorflow:The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
WARNING:tensorflow:Gradients do not exist for variables ['tf_roberta_model/roberta/pooler/dense/kernel:0', 'tf_roberta_model/roberta/pooler/dense/bias:0'] when minimizing the loss. If you're using `model.compile()`, did you forget to provide a `loss`argument?
15287/15287 [==============================] - ETA: 0s - loss: 3.9956 - f1_score: 0.0804WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
WARNING:tensorflow:The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
15287/15287 [==============================] - 2651s 172ms/step - loss: 3.9956 - f1_score: 0.0804 - val_loss: 3.9584 - val_f1_score: 0.0682 - lr: 0.0020
Epoch 2/30
 1867/15287 [==>...........................] - ETA: 36:53 - loss: 3.9015 - f1_score: 0.0788

 이것도 이런식의 정확도인데 ..

"""

Epoch 1/30
15287/15287 [==============================] - 2874s 187ms/step - loss: 4.4376 - f1_score: 0.0797 - val_loss: 4.0755 - val_f1_score: 0.0682 - lr: 0.0200
Epoch 2/30
15287/15287 [==============================] - 2822s 185ms/step - loss: 4.2109 - f1_score: 0.0793 - val_loss: 4.0376 - val_f1_score: 0.0682 - lr: 0.0200
Epoch 3/30
15287/15287 [==============================] - 2829s 185ms/step - loss: 4.1968 - f1_score: 0.0798 - val_loss: 4.1564 - val_f1_score: 0.0041 - lr: 0.0200
Epoch 4/30
15287/15287 [==============================] - 2824s 185ms/step - loss: 4.1964 - f1_score: 0.0792 - val_loss: 4.1189 - val_f1_score: 0.0682 - lr: 0.0200
Epoch 5/30
15287/15287 [==============================] - 2839s 186ms/step - loss: 4.1927 - f1_score: 0.0802 - val_loss: 4.0865 - val_f1_score: 0.0682 - lr: 0.0200
Epoch 6/30
15287/15287 [==============================] - 2812s 184ms/step - loss: 4.1913 - f1_score: 0.0794 - val_loss: 4.1428 - val_f1_score: 0.0020 - lr: 0.0200
Epoch 7/30
15287

KeyboardInterrupt: 

In [ ]:
model.load_weights(checkpoint_path)
val_weighted_f1 = model.evaluate(val_ds)[1]
print(f"val_weighted_f1: {val_weighted_f1}")


## Inference

In [ ]:
test_ds = DataGenerator(
    X_test_txt.values,None,
    batch_size=1,
    shuffle=False,
    include_targets=False
)


In [ ]:
pred_prob =[]

for i in range(test_ds.__len__()):
    pred_prob.append(model.predict(test_ds.__getitem__(i)))
pred_prob = np.vstack(pred_prob)
pred = np.argmax(pred_prob, axis=1)

y_decoder = {value : key for key, value in y_encoder.items()}
result = np.array([y_decoder[v] for v in pred])


pd.Series(result).value_counts()


In [ ]:
submission = pd.read_csv("input/sample_submission.csv")
submission['cat3'] = result
submission.to_csv(f'{parser.description}.csv',index=False)